In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/next-word-prediction/1661-0.txt


# Dependencies

In [56]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM,Embedding,Dense,Dropout,BatchNormalization, GRU
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
from tensorflow.keras.models import Sequential
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from tensorflow.keras.utils import to_categorical

In [3]:
import numpy as np

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Load text

In [5]:
file_path = '/kaggle/input/next-word-prediction/1661-0.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    text = "".join(lines)

print(text[:200])

﻿
Project Gutenberg's The Adventures of Sherlock Holmes, by Arthur Conan Doyle

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, gi


In [6]:
cleaned_text = re.sub(r'[^\x00-\x7F]+', '', text)
text = cleaned_text.replace('\n',' ').replace('\r','')
text = re.sub('\s+',' ',text).strip()
text[:100]

"Project Gutenberg's The Adventures of Sherlock Holmes, by Arthur Conan Doyle This eBook is for the u"

# Tokenization

In [7]:
sentences = sent_tokenize(text)
print(sentences[:10])

["Project Gutenberg's The Adventures of Sherlock Holmes, by Arthur Conan Doyle This eBook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever.", 'You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.net Title: The Adventures of Sherlock Holmes Author: Arthur Conan Doyle Release Date: November 29, 2002 [EBook #1661] Last Updated: May 20, 2019 Language: English Character set encoding: UTF-8 *** START OF THIS PROJECT GUTENBERG EBOOK THE ADVENTURES OF SHERLOCK HOLMES *** Produced by an anonymous Project Gutenberg volunteer and Jose Menendez cover The Adventures of Sherlock Holmes by Arthur Conan Doyle Contents I.', 'A Scandal in Bohemia II.', 'The Red-Headed League III.', 'A Case of Identity IV.', 'The Boscombe Valley Mystery V. The Five Orange Pips VI.', 'The Man with the Twisted Lip VII.', 'The Adventure of the Blue Carbuncle VIII.', 'The Adventure of the Speckle

In [8]:
tokens = set()
for sent in sentences : 
    token = word_tokenize(sent)
    tokens.update(token)

vocab_len = len(tokens)
print (vocab_len)

9484


In [9]:
word_index = {word: index for index,word in enumerate(tokens)}
index_word = {index : word for index,word in enumerate(tokens)}

In [10]:
def WordSequenceToNumberSequence(words) :
    ns = []
    for word in words :
        ns.append(word_index[word])

    return ns

In [11]:
sequences=[]
for sent in sentences :
    words = word_tokenize(sent)
    idx = WordSequenceToNumberSequence(words)
    for i in range (1,len(words)) :
        sequence = idx [:i]
        sequences.append(sequence)


print(sequences[:10])

[[2398], [2398, 1113], [2398, 1113, 871], [2398, 1113, 871, 1756], [2398, 1113, 871, 1756, 2393], [2398, 1113, 871, 1756, 2393, 2791], [2398, 1113, 871, 1756, 2393, 2791, 5920], [2398, 1113, 871, 1756, 2393, 2791, 5920, 8115], [2398, 1113, 871, 1756, 2393, 2791, 5920, 8115, 4817], [2398, 1113, 871, 1756, 2393, 2791, 5920, 8115, 4817, 4365]]


In [12]:
max_len= max([len(x) for x in sequences])
max_len

122

## **Padding**

In [13]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

# **Input / Target split**

In [14]:
input_sequences =[x[:max_len-1] for x in sequences]
targets = [x[-1] for x in sequences]

In [15]:
targets=np.array(targets)
input_sequences=np.array(input_sequences)
targets.shape

(115801,)

In [16]:
input_sequences.shape

(115801, 121)

## Build LSTM model

In [17]:
model = Sequential()
model.add(Embedding(vocab_len, 64, input_length=max_len-1))
model.add(LSTM(100))
model.add(Dense(vocab_len, activation="softmax"))



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1758296342.797987      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1758296342.798670      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [18]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [19]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [21]:
model.fit(input_sequences, targets, epochs=50, verbose=1)

Epoch 1/50
3619/3619 ━━━━━━━━━━━━━━━━━━━━ 29s 8ms/step - accuracy: 0.3388 - loss: 3.1706
Epoch 2/50
3619/3619 ━━━━━━━━━━━━━━━━━━━━ 29s 8ms/step - accuracy: 0.3598 - loss: 3.0380
Epoch 3/50
3619/3619 ━━━━━━━━━━━━━━━━━━━━ 29s 8ms/step - accuracy: 0.3784 - loss: 2.9251
Epoch 4/50
3619/3619 ━━━━━━━━━━━━━━━━━━━━ 29s 8ms/step - accuracy: 0.3983 - loss: 2.8120
Epoch 5/50
3619/3619 ━━━━━━━━━━━━━━━━━━━━ 29s 8ms/step - accuracy: 0.4156 - loss: 2.7171
Epoch 6/50
3619/3619 ━━━━━━━━━━━━━━━━━━━━ 29s 8ms/step - accuracy: 0.4342 - loss: 2.6174
Epoch 7/50
3619/3619 ━━━━━━━━━━━━━━━━━━━━ 29s 8ms/step - accuracy: 0.4454 - loss: 2.5504
Epoch 8/50
3619/3619 ━━━━━━━━━━━━━━━━━━━━ 29s 8ms/step - accuracy: 0.4624 - loss: 2.4695
Epoch 9/50
3619/3619 ━━━━━━━━━━━━━━━━━━━━ 29s 8ms/step - accuracy: 0.4778 - loss: 2.3935
Epoch 10/50
3619/3619 ━━━━━━━━━━━━━━━━━━━━ 29s 8ms/step - accuracy: 0.4880 - loss: 2.3314
Epoch 11/50
3619/3619 ━━━━━━━━━━━━━━━━━━━━ 29s 8ms/step - accuracy: 0.4994 - loss: 2.2697
Epoch 12/50
3619/36

In [45]:
model.fit(input_sequences, targets, epochs=10, verbose=1)

Epoch 1/10
3619/3619 ━━━━━━━━━━━━━━━━━━━━ 29s 8ms/step - accuracy: 0.7052 - loss: 1.3038
Epoch 2/10
3619/3619 ━━━━━━━━━━━━━━━━━━━━ 29s 8ms/step - accuracy: 0.7066 - loss: 1.2917
Epoch 3/10
3619/3619 ━━━━━━━━━━━━━━━━━━━━ 29s 8ms/step - accuracy: 0.7084 - loss: 1.2841
Epoch 4/10
3619/3619 ━━━━━━━━━━━━━━━━━━━━ 28s 8ms/step - accuracy: 0.7102 - loss: 1.2777
Epoch 5/10
3619/3619 ━━━━━━━━━━━━━━━━━━━━ 28s 8ms/step - accuracy: 0.7127 - loss: 1.2672
Epoch 6/10
3619/3619 ━━━━━━━━━━━━━━━━━━━━ 29s 8ms/step - accuracy: 0.7106 - loss: 1.2662
Epoch 7/10
3619/3619 ━━━━━━━━━━━━━━━━━━━━ 28s 8ms/step - accuracy: 0.7150 - loss: 1.2516
Epoch 8/10
3619/3619 ━━━━━━━━━━━━━━━━━━━━ 29s 8ms/step - accuracy: 0.7164 - loss: 1.2431
Epoch 9/10
3619/3619 ━━━━━━━━━━━━━━━━━━━━ 29s 8ms/step - accuracy: 0.7180 - loss: 1.2389
Epoch 10/10
3619/3619 ━━━━━━━━━━━━━━━━━━━━ 29s 8ms/step - accuracy: 0.7184 - loss: 1.2368


## Build GRU Model

In [57]:

modelGRU = Sequential()
modelGRU.add(Embedding(vocab_len, 64, input_length=max_len-1))
modelGRU.add(GRU(100))  # <-- GRU
modelGRU.add(Dense(vocab_len, activation="softmax"))
modelGRU.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [58]:
modelGRU.fit(input_sequences, targets, epochs=50, verbose=1)

Epoch 1/50
3619/3619 ━━━━━━━━━━━━━━━━━━━━ 30s 8ms/step - accuracy: 0.0988 - loss: 6.3496
Epoch 2/50
3619/3619 ━━━━━━━━━━━━━━━━━━━━ 27s 8ms/step - accuracy: 0.1650 - loss: 5.1677
Epoch 3/50
3619/3619 ━━━━━━━━━━━━━━━━━━━━ 27s 8ms/step - accuracy: 0.1905 - loss: 4.7359
Epoch 4/50
3619/3619 ━━━━━━━━━━━━━━━━━━━━ 27s 8ms/step - accuracy: 0.2117 - loss: 4.4201
Epoch 5/50
3619/3619 ━━━━━━━━━━━━━━━━━━━━ 28s 8ms/step - accuracy: 0.2305 - loss: 4.1251
Epoch 6/50
3619/3619 ━━━━━━━━━━━━━━━━━━━━ 27s 8ms/step - accuracy: 0.2532 - loss: 3.8718
Epoch 7/50
3619/3619 ━━━━━━━━━━━━━━━━━━━━ 27s 8ms/step - accuracy: 0.2770 - loss: 3.6419
Epoch 8/50
3619/3619 ━━━━━━━━━━━━━━━━━━━━ 27s 8ms/step - accuracy: 0.3021 - loss: 3.4373
Epoch 9/50
3619/3619 ━━━━━━━━━━━━━━━━━━━━ 27s 8ms/step - accuracy: 0.3323 - loss: 3.2498
Epoch 10/50
3619/3619 ━━━━━━━━━━━━━━━━━━━━ 27s 8ms/step - accuracy: 0.3585 - loss: 3.0735
Epoch 11/50
3619/3619 ━━━━━━━━━━━━━━━━━━━━ 27s 8ms/step - accuracy: 0.3876 - loss: 2.9169
Epoch 12/50
3619/36

# Make Predictions

In [48]:
def predict_word(seed_text, model=model):
    seed_text = word_tokenize(seed_text)
    seed_sequence = WordSequenceToNumberSequence(seed_text)
    padded_seq = pad_sequences([seed_sequence], maxlen=max_len-1, padding='pre')
    prediction = model.predict(padded_seq)
    prediction_index = np.argmax(prediction)
    prediction_word = index_word[prediction_index]
    return prediction_word

In [52]:
text = "My name "
try : 
    while not text[-1]=="." and len(text)<100:
        text += predict_word(text)+ " "
        print(text)
except exception :
    print ("probably the last word doesnt exist in the dictionary")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
My name is 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
My name is Helen 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
My name is Helen Stoner 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
My name is Helen Stoner , 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
My name is Helen Stoner , and 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
My name is Helen Stoner , and we 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
My name is Helen Stoner , and we shall 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
My name is Helen Stoner , and we shall see 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
My name is Helen Stoner , and we shall see how 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
My name is Helen Stoner , and we shall see how they 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
My name is Helen Stoner , and we shall see how they are 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
My name is Helen Stoner , and we shall see how they are attained 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
My name is Helen Stoner , and we shall see how they

In [60]:
text = "My name "
try : 
    while not text[-1]=="." and len(text)<110:
        text += predict_word(text, model= modelGRU)+ " "
        print(text)
except exception :
    print ("probably the last word doesnt exist in the dictionary")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
My name is 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
My name is John 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
My name is John Robinson 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
My name is John Robinson , 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
My name is John Robinson , he 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
My name is John Robinson , he answered 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
My name is John Robinson , he answered with 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
My name is John Robinson , he answered with a 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
My name is John Robinson , he answered with a sidelong 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
My name is John Robinson , he answered with a sidelong glance 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
My name is John Robinson , he answered with a sidelong glance of 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
My name is John Robinson , he answered with a sidelong glance of our 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
My na